In [15]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import utils
from keras.layers import Input, Dense, Activation,  Flatten, Dropout, Lambda, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.models import Model
from resnet50 import Resnet50
from __future__ import print_function

In [2]:
bdir="/home/ubuntu/nbs/courses/deeplearning1/nbs/data/fishery/"
trdir=bdir + "train/"
tedir=bdir + "test_stg1/"
vdir = bdir + "valid/"

In [28]:
??utils.get_classes

### One time setup to move random train images into validation set

In [3]:
import glob, os

In [4]:
def moveallimages(src, pattern, dest):
    imgs=glob.glob(src + "/" + pattern)
    for img in imgs:
        os.rename(img, dest + "/" + os.path.basename(img))

# Function to copy/move nimages random images from 'src' to 'dest'.
from shutil import copyfile        
def moverandimages(src, nimages, dest, docopy=0):
    imgs=glob.glob(src + "/" + "*.jpg")
    shuff = np.random.permutation(imgs)
    for i in range(nimages):
        if (docopy == 0):
            os.rename(shuff[i], dest + "/" + os.path.basename(shuff[i]))
        else:
            copyfile(shuff[i], dest + "/" + os.path.basename(shuff[i]))
            
# Create directories in valid directory similar to train directory followed by copying images
for src in glob.glob(trdir + "/*"):
    dest=vdir + os.path.basename(src)
    os.mkdir(dest)
    moverandimages(src, 25, dest, docopy=1)

### Analyse training images

In [11]:
import PIL, glob

classes = [os.path.basename(f) for f in glob.glob(trdir + "/*")]
for c in classes:
    imgsz = [PIL.Image.open(f).size for f in glob.glob(trdir + "/" + c + "/*")] 
    uniqimgsz = list(set(imgsz))
    print("class=" + c + " imgsz=", uniqimgsz)

class=DOL imgsz= [(1280, 974), (1280, 720), (1280, 750)]
class=YFT imgsz= [(1280, 750), (1244, 700), (1192, 670), (1280, 720), (1276, 718), (1280, 974), (1518, 854), (1280, 924)]
class=BET imgsz= [(1280, 974), (1244, 700), (1192, 670), (1280, 720), (1276, 718), (1280, 750), (1518, 854)]
class=LAG imgsz= [(1280, 720)]
class=OTHER imgsz= [(1280, 750), (1280, 720), (1280, 974)]
class=NoF imgsz= [(1280, 750), (1244, 700), (1732, 974), (1334, 750), (1192, 670), (1280, 720), (1276, 718), (1518, 854)]
class=ALB imgsz= [(1280, 974), (1244, 700), (1192, 670), (1280, 720), (1276, 718), (1280, 750), (1518, 854), (1280, 924)]
class=SHARK imgsz= [(1280, 750), (1280, 720), (1280, 974)]


### Generate features from Resnet without last FC layer

In [5]:
rm = Resnet50(size=(640, 360), include_top=False)

In [33]:
trn_batches = utils.get_batches(trdir, target_size=(640, 360), shuffle=False)
val_batches = utils.get_batches(vdir, target_size=(640, 360), shuffle=False)

Found 3777 images belonging to 8 classes.
Found 200 images belonging to 8 classes.


In [39]:
val_features = rm.model.predict_generator(val_batches, val_batches.nb_sample)

In [40]:
trn_features = rm.model.predict_generator(trn_batches, trn_batches.nb_sample)

In [41]:
utils.save_array(bdir + "/results/resnet_val_640_360.dat", val_features)
utils.save_array(bdir + "/results/resnet_trn_640_360.dat", trn_features)

In [42]:
val_labels = utils.onehot(val_batches.classes)
trn_labels = utils.onehot(trn_batches.classes)
utils.save_array(bdir + "/results/resnet_val_labels_640_360.dat", val_labels)
utils.save_array(bdir + "/results/resnet_trn_labels_640_360.dat", trn_labels)

#### Load convolution features from disk in case it was generated in a previous run

In [4]:
val_features = utils.load_array(bdir + "/results/resnet_val_640_360.dat")
trn_features = utils.load_array(bdir + "/results/resnet_trn_640_360.dat")
val_labels = utils.load_array(bdir + "/results/resnet_val_labels_640_360.dat")
trn_labels = utils.load_array(bdir + "/results/resnet_trn_labels_640_360.dat")

### Create classifier for resnet convolution output

In [41]:
i = Input(shape=rm.model.output_shape[1:])

l = GlobalAveragePooling2D(input_shape=rm.model.output_shape[1:])(i)
l = Dense(2048, activation='relu')(l)
l = BatchNormalization()(l)
l = Dropout(0.5)(l)
l = Dense(1024, activation='relu')(l)
l = BatchNormalization()(l)
l = Dropout(0.5)(l)
l = Dense(8, activation='softmax')(l)
m = Model(input=i, output=l)

th


In [45]:
m.compile(optimizer=Adam(1e-4), metrics=['accuracy'], loss='categorical_crossentropy')
m.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_15 (InputLayer)            (None, 2048, 20, 12)  0                                            
____________________________________________________________________________________________________
globalaveragepooling2d_14 (Global(None, 2048)          0           input_15[0][0]                   
____________________________________________________________________________________________________
dense_33 (Dense)                 (None, 2048)          4196352     globalaveragepooling2d_14[0][0]  
____________________________________________________________________________________________________
batchnormalization_19 (BatchNorma(None, 2048)          4096        dense_33[0][0]                   
___________________________________________________________________________________________

In [61]:
m.optimizer.lr = 1e-5
m.fit(trn_features, trn_labels, batch_size=64, nb_epoch=10, validation_data=(val_features, val_labels))

Train on 3777 samples, validate on 200 samples
Epoch 1/10
3777/3777 [==============================] - 4s - loss: 0.0726 - acc: 0.9780 - val_loss: 0.0112 - val_acc: 0.9950
Epoch 2/10
3777/3777 [==============================] - 4s - loss: 0.0783 - acc: 0.9735 - val_loss: 0.0267 - val_acc: 0.9900
Epoch 3/10
3777/3777 [==============================] - 4s - loss: 0.0743 - acc: 0.9778 - val_loss: 0.0048 - val_acc: 1.0000
Epoch 4/10
3777/3777 [==============================] - 4s - loss: 0.0662 - acc: 0.9793 - val_loss: 0.0066 - val_acc: 0.9950
Epoch 5/10
3777/3777 [==============================] - 4s - loss: 0.0638 - acc: 0.9767 - val_loss: 0.0038 - val_acc: 1.0000
Epoch 6/10
3777/3777 [==============================] - 4s - loss: 0.0583 - acc: 0.9801 - val_loss: 0.0086 - val_acc: 0.9950
Epoch 7/10
3777/3777 [==============================] - 4s - loss: 0.0731 - acc: 0.9759 - val_loss: 0.0417 - val_acc: 0.9850
Epoch 8/10
3777/3777 [==============================] - 4s - loss: 0.0716 - ac

In [50]:
m.save_weights(bdir + "/results/resnet_3layer_dense_20epoch_dropout_weights.dat")

### Generate test results

In [51]:
tst_batches=utils.get_batches(tedir, target_size=(640, 360), shuffle=False)
tst_conv_features = rm.model.predict_generator(tst_batches, tst_batches.nb_sample)

Found 1000 images belonging to 1 classes.


In [62]:
tst_labels = m.predict(tst_conv_features, batch_size=128)
tst_labels = np.clip(tst_labels, 0.01, 0.99)

In [63]:
import pandas as pd

class_labels=sorted(trn_batches.class_indices)
results = pd.DataFrame(columns=class_labels, data=tst_labels)
bfnames = [f[8:] for f in tst_batches.filenames]
results.insert(0, 'image', bfnames)

In [64]:
results.to_csv(bdir + "submission_640_360_resnet_3lrd_dropout_99clip.csv", index=False)